In [1]:
import math
import os
import shutil
import time
import numpy as np
from datetime import datetime
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
import torchvision.models as models
from HSI_class import HSI
import createSample as CS
import augmentation as aug

import simsiam.loader

import random

start_time = time.time()

# Check if GPU is available
print("GPU Available:", torch.cuda.is_available())

# If available, print the GPU name
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    
sample_per_class = 5
num_per_category_augment_1 = 10
num_per_category_augment_2 = 10
epochs = 200

batch_size = 20
test_size = 0.5

random_indice = 1

seeded_run = True
seed = 10


GPU Available: True
GPU Name: NVIDIA GeForce RTX 4080 SUPER


In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # PyTorch determinism
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)

if seeded_run:
    set_seed(seed)
    print("seed has been set")
    print(f"seet used: {seed}")

seed has been set
seet used: 10


In [3]:
dataset_path = r"D:\FathanAbi\tugas-akhir-model-deteksi-tumpahan-minyakl\Hyperspectral oil spill detection datasets"
# dataset_path = r"C:\Users\Asus TUF\Documents\code\TA\Hyperspectral oil spill detection datasets"

dataset = []

i = 0
for filename in os.listdir(dataset_path):
    if i > 0:
        break
    file_path = os.path.join(dataset_path, filename)
    if os.path.isfile(file_path):  # Check if it's a file
        print(f"Processing file: {file_path}")
        hsi = HSI(file_path)
        dataset.append(hsi)
    i += 1

hsi_ = dataset[0]
patch_size = 9
sample_per_class = sample_per_class

indices_0 = []
indices_1 = []

print(f"random: {random_indice}")

if random_indice:
    print("generating random sample")
    selected_patch_0, selected_patch_1, indices_0, indices_1 = CS.createSample(hsi_, patch_size, sample_per_class)
else:
    print("using generated indices")
    indices_0 = [(np.int64(188), np.int64(124)), (np.int64(523), np.int64(150)), (np.int64(1003), np.int64(474)), (np.int64(616), np.int64(508)), (np.int64(905), np.int64(552))]
    indices_1 = [(np.int64(106), np.int64(606)), (np.int64(297), np.int64(468)), (np.int64(926), np.int64(35)), (np.int64(536), np.int64(519)), (np.int64(508), np.int64(442))]

    selected_patch_0, selected_patch_1 = CS.getSample(hsi_, patch_size, sample_per_class, indices_0, indices_1)


i =0
half_patch = patch_size // 2

indices = indices_0 +  indices_1

# Concatenating along axis 0
x_train = np.concatenate((selected_patch_0, selected_patch_1), )

y_train = np.array([])

gt = hsi_.gt
for indice in indices:
    # print(gt[indice[0]][indice[1]])
    y_train = np.append(y_train, gt[indice[0]][indice[1]])

count = np.count_nonzero(y_train == 0)  # Count elements equal to 0
print(f'number of element equal 0 {count}')

count = np.count_nonzero(y_train == 1)  # Count elements equal to 1
print(f'number of element equal 1 {count}')



# Print shape to verify
print(f"x_train shape: {x_train.shape}")  # Expected output: (10, 9, 9, 224)
print(f"y_train shape: {y_train.shape}") 




Processing file: D:\FathanAbi\tugas-akhir-model-deteksi-tumpahan-minyakl\Hyperspectral oil spill detection datasets\GM01.mat
random: 1
generating random sample
hsi shape
(1243, 684, 224)
creating 5 Randomly chosen 0 indices:
creating 5 Randomly chosen 1 indices:
indices 0 used: [(np.int64(910), np.int64(192)), (np.int64(51), np.int64(255)), (np.int64(689), np.int64(202)), (np.int64(772), np.int64(547)), (np.int64(920), np.int64(471))]
indices 1 used: [(np.int64(22), np.int64(455)), (np.int64(170), np.int64(145)), (np.int64(410), np.int64(233)), (np.int64(1055), np.int64(123)), (np.int64(469), np.int64(582))]
number of element equal 0 5
number of element equal 1 5
x_train shape: (10, 9, 9, 224)
y_train shape: (10,)


In [4]:
i =1
half_patch = patch_size // 2
print(hsi_.img[indices_0[i][0]][indices_0[i][1]])
print(selected_patch_0[i][half_patch][half_patch])

print(hsi_.img[indices_1[i][0]][indices_1[i][1]])
print(selected_patch_1[i][half_patch][half_patch])
i =4
half_patch = patch_size // 2
print(hsi_.img[indices_0[i][0]][indices_0[i][1]])
print(selected_patch_0[i][half_patch][half_patch])

print(hsi_.img[indices_1[i][0]][indices_1[i][1]])
print(selected_patch_1[i][half_patch][half_patch])

[-219 -372  361  395  442  586  641  667  632  594  559  517  484  453
  432  402  375  347  324  297  275  250  230  202  176  163  153  142
  137  125  124  116  116  111   99   86   73   65   54   19   39   47
   59   20   40   41   37   35   33   17   19   23   38   34   35   27
   28   11    1   -6   -5  -13 -184  -77    5    4   19   25   26   29
   31   25   27   27   28   23   18   15    4  -26 -217 -248 -115 -122
  -43   -5    2    0    4    0    6   12   17   17   22   22   23   22
   36   17   18    7    4   15   -5  -31    0    0    0    0    0    0
    0    0  -41 -185  -52  -35  -86  -44   -6    6   12   12   22   22
   20   20   22   25   27   26   24   21   20   24   27   16   23   28
   25   29   22   18   12   17   14    5    7   -1    5  120    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
  -66  -65    0    0    1   -9    2  -15   13   14   17   18   15   12
   18   14   17   22   21   18   17   20   27   10   11   13   12   16
    8 

In [5]:
n_category = 2
band_size = 224
num_per_category_augment_1 = num_per_category_augment_1
num_per_category_augment_2 = num_per_category_augment_2

data_augment1, label_augment1 = aug.Augment_data(x_train, y_train, n_category, patch_size, band_size, num_per_category_augment_1)

data_augment2, label_augment2 = aug.Augment_data2(x_train, y_train, n_category, patch_size, band_size, num_per_category_augment_2)

print(f"hasil augmentasi 1 shape: {data_augment1.shape}")
print(f"label augmentai 1 shape: {label_augment1.shape}")

print(f"hasil augmentasi 2 shape: {data_augment2.shape}")
print(f"label augmentasi 2 shape: {label_augment2.shape}")

print(label_augment1)
print(label_augment2)

# # Count occurrences of each unique element
# counts1 = np.bincount(label_augment1)

# # Print results
# for i, count in enumerate(counts1):
#     print(f"Element {i} occurs {count} times.")

# counts2 = np.bincount(label_augment2)

# # Print results
# for i, count in enumerate(counts2):
#     print(f"Element {i} occurs {count} times.")

# print(label_augment1[3])

data_augment = np.concatenate((data_augment1, data_augment2))
label_augment = np.concatenate((label_augment1, label_augment2))

print(f"hasil augmentasi gabungan untuk training: {data_augment.shape}")
print(f"label augmentasi gabungan: {label_augment.shape}")

# print(label_augment)

# Count occurrences of each unique element
counts = np.bincount(label_augment)

# Print results
for i, count in enumerate(counts):
    print(f"Element {i} occurs {count} times.")

hasil augmentasi 1 shape: (20, 9, 9, 224)
label augmentai 1 shape: (20,)
hasil augmentasi 2 shape: (20, 9, 9, 224)
label augmentasi 2 shape: (20,)
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
hasil augmentasi gabungan untuk training: (40, 9, 9, 224)
label augmentasi gabungan: (40,)
Element 0 occurs 20 times.
Element 1 occurs 20 times.


In [15]:
class SimSiam(nn.Module):
    """
    Build a SimSiam model.
    """
    def __init__(self, base_encoder, dim=2048, pred_dim=512):
        """
        dim: feature dimension (default: 2048)
        pred_dim: hidden dimension of the predictor (default: 512)
        """
        super(SimSiam, self).__init__()
        # Custom Convolutional Layer: Process 9x9x224 input
        self.pre_conv = nn.Sequential(
            nn.Conv2d(in_channels=224, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.AdaptiveAvgPool2d((1, 1))  # Reduce to (256, 1, 1)
        )
        # Fully Connected Layer to reshape to (64, 56, 56)
        self.fc = nn.Linear(256 * 1 * 1, 64 * 56 * 56)

        # create the encoder
        # num_classes is the output fc dimension, zero-initialize last BNs
        # self.encoder = base_encoder(num_classes=dim, zero_init_residual=True)
        self.encoder = base_encoder(pretrained=True)

        self.encoder.features = nn.Sequential(*list(self.encoder.features.children())[1:])

        # Modify the classifier to match the desired output dimensions
        self.encoder.classifier[6] = nn.Linear(4096, dim)

        # Fix: Get the correct input dimension from VGG16 classifier
        prev_dim = self.encoder.classifier[6].out_features

        # Fix: Assign modified layers to classifier instead of non-existing 'fc'
        self.projector = nn.Sequential(nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # first layer
                                        nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # second layer
                                        # self.projector,
                                        nn.BatchNorm1d(dim, affine=False)) # output layer
                                        

        # self.projector[6].bias.requires_grad = False

        # build a 3-layer projector
        # prev_dim = self.encoder.fc.weight.shape[1]
        # self.encoder.fc = nn.Sequential(nn.Linear(prev_dim, prev_dim, bias=False),
        #                                 nn.BatchNorm1d(prev_dim),
        #                                 nn.ReLU(inplace=True), # first layer
        #                                 nn.Linear(prev_dim, prev_dim, bias=False),
        #                                 nn.BatchNorm1d(prev_dim),
        #                                 nn.ReLU(inplace=True), # second layer
        #                                 self.encoder.fc,
        #                                 nn.BatchNorm1d(dim, affine=False)) # output layer
        # self.encoder.fc[6].bias.requires_grad = False # hack: not use bias as it is followed by BN

        # build a 2-layer predictor
        self.predictor = nn.Sequential(nn.Linear(dim, pred_dim, bias=False),
                                        nn.BatchNorm1d(pred_dim),
                                        nn.ReLU(inplace=True), # hidden layer
                                        nn.Linear(pred_dim, dim)) # output layer

    def forward(self, x1, x2):
        """
        Input:
            x1: first views of images
            x2: second views of images
        Output:
            p1, p2, z1, z2: predictors and targets of the network
            See Sec. 3 of https://arxiv.org/abs/2011.10566 for detailed notations
        """
        print(f"x1 before preconv {x1.shape}")
        x1 = self.pre_conv(x1)
        x2 = self.pre_conv(x2)
        print(f"x1 after preconv {x1.shape}")

        print(f"x1 before flatten {x1.shape}")
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)
        print(f"x1 after flatten {x1.shape}")

        print(f"x1 before FC {x1.shape}")
        x1 = self.fc(x1)
        x2 = self.fc(x2)
        print(f"x1 after FC {x1.shape}")

        print(f"x1 after reshape {x1.shape}")
        x1 = x1.view(x1.size(0), 64, 56, 56)
        x2 = x2.view(x2.size(0), 64, 56, 56)
        print(f"x1 after reshape {x1.shape}")
        # compute features for one view

        # print(x1.shape)
        print(f"x1 before vgg.features {x1.shape}")
        z1 = self.encoder.features(x1) # NxC
        z2 = self.encoder.features(x2) # NxC
        print(f"z1 after vgg.features {z1.shape}")

        print(f"z1 before vgg.avgpool {z1.shape}")
        z1 = self.encoder.avgpool(z1)
        z2 = self.encoder.avgpool(z2)
        print(f"z1 after vgg.avgpool {z1.shape}")


        z1 = torch.flatten(z1, 1)
        z2 = torch.flatten(z2, 1)
        print(f"z1 after vgg.flatten {z1.shape}")

        print(f"z1 before vgg.classifier {z1.shape}")
        z1 = self.encoder.classifier(z1)
        z2 = self.encoder.classifier(z2)
        print(f"z1 after vgg.classifier {z1.shape}")
        # print(z1.shape)
        
        print(f"z1 before projector {z1.shape}")
        z1 = self.projector(z1)
        z2 = self.projector(z2)
        print(f"z1 after projector {z1.shape}")

        print(f"z1 before predictor {z1.shape}")
        p1 = self.predictor(z1) # NxC
        p2 = self.predictor(z2) # NxC
        print(f"p1 after predictor {z1.shape}")

        return p1, p2, z1.detach(), z2.detach()

In [18]:
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

print(model_names)
# create model
arch = 'vgg16' 
print("=> creating model '{}'".format(arch))
model = SimSiam(
    models.__dict__[arch])


lr = 0.01
init_lr = lr * batch_size / 256
gpu = 0

print(model)

['alexnet', 'convnext_base', 'convnext_large', 'convnext_small', 'convnext_tiny', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'efficientnet_v2_l', 'efficientnet_v2_m', 'efficientnet_v2_s', 'get_model', 'get_model_builder', 'get_model_weights', 'get_weight', 'googlenet', 'inception_v3', 'list_models', 'maxvit_t', 'mnasnet0_5', 'mnasnet0_75', 'mnasnet1_0', 'mnasnet1_3', 'mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small', 'regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf', 'regnet_x_400mf', 'regnet_x_800mf', 'regnet_x_8gf', 'regnet_y_128gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf', 'regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'resnext101_32x8d', 'resnext101_64x4d', 'resnext50_32x4d', 'shufflenet_v2_x0_

In [19]:
test = data_augment[0]
test = torch.tensor(test)
test = test.to(torch.float32)
test = test.unsqueeze(0)

input = test
input = input.permute(0, 3, 1, 2)

test2 = data_augment[1]
test2 = torch.tensor(test2)
test2 = test2.to(torch.float32)
test2 = test2.unsqueeze(0)

input2 = test2
input2 = input2.permute(0, 3, 1, 2)

print(f"input shape: {input.shape}")
print(f"input2 shape: {input2.shape}")

# Pass the input through the model
model.eval()
p1, p2, z1, z2  = model(input, input2)

print(p1)
print(p2)
print(z1)
print(z2)

input shape: torch.Size([1, 224, 9, 9])
input2 shape: torch.Size([1, 224, 9, 9])
x1 before preconv torch.Size([1, 224, 9, 9])
x1 after preconv torch.Size([1, 256, 1, 1])
x1 before flatten torch.Size([1, 256, 1, 1])
x1 after flatten torch.Size([1, 256])
x1 before FC torch.Size([1, 256])
x1 after FC torch.Size([1, 200704])
x1 after reshape torch.Size([1, 200704])
x1 after reshape torch.Size([1, 64, 56, 56])
x1 before vgg.features torch.Size([1, 64, 56, 56])
z1 after vgg.features torch.Size([1, 512, 1, 1])
z1 before vgg.avgpool torch.Size([1, 512, 1, 1])
z1 after vgg.avgpool torch.Size([1, 512, 7, 7])
z1 after vgg.flatten torch.Size([1, 25088])
z1 before vgg.classifier torch.Size([1, 25088])
z1 after vgg.classifier torch.Size([1, 2048])
z1 before projector torch.Size([1, 2048])
z1 after projector torch.Size([1, 2048])
z1 before predictor torch.Size([1, 2048])
p1 after predictor torch.Size([1, 2048])
tensor([[ 0.0002,  0.0215,  0.0078,  ..., -0.0418, -0.0045, -0.0472]],
       grad_fn=<Add

In [20]:
criterion = nn.CosineSimilarity(dim=1).cuda(gpu)
print(gpu)
optim_params = model.parameters()

momentum = 0.9
weight_decay = 1e-4

optimizer = torch.optim.SGD(optim_params, init_lr,
                                momentum=momentum,
                                weight_decay=weight_decay)

cudnn.benchmark = True
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

augmentation = [
    transforms.RandomHorizontalFlip(),  # Flip along width
    transforms.RandomVerticalFlip(),    # Flip along height
    transforms.RandomRotation(20),      # Rotate image slightly
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize hyperspectral data
]

transform = simsiam.loader.TwoCropsTransform(transforms.Compose(augmentation))

print(data_augment.shape)

0
(40, 9, 9, 224)


In [21]:
pretrain_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Pretrain Parameter {pretrain_parameters}")

Pretrain Parameter 205281472


In [22]:
from torchvision.models import vgg16

class VGG16_HSI(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16_HSI, self).__init__()

         # Custom Convolutional Layer: Process 9x9x224 input
        self.pre_conv = nn.Sequential(
            nn.Conv2d(in_channels=224, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.AdaptiveAvgPool2d((1, 1))  # Reduce to (256, 1, 1)
        )

        # Fully Connected Layer to reshape to (64, 56, 56)
        self.fc = nn.Linear(256 * 1 * 1, 64 * 56 * 56)

        # Load VGG-16 Model
        self.encoder = vgg16(pretrained=False)

        # Remove first VGG-16 conv layer
        self.encoder.features = nn.Sequential(*list(self.encoder.features.children())[1:])

        # Modify classifier to output 2 classes
        self.encoder.classifier[6] = nn.Linear(4096, 2)

    def forward(self, x):
        # print(f'before {x.shape}')
        x = self.pre_conv(x)  # Process hyperspectral input
        x = x.view(x.size(0), -1)  # Flatten

        # print(f'after preconv {x.shape}')
        x = self.fc(x)  # Fully connected layer
        # print(f'after fc {x.shape}')
        # Reshape to (batch_size, 64, 56, 56) before passing to VGG
        x = x.view(x.size(0), 64, 56, 56)
        # print(f'after reshape, before vgg second layer {x.shape}')

        x = self.encoder.features(x)  # Pass to VGG-16
        x = self.encoder.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.encoder.classifier(x)  # Final classification layer

        return x

In [25]:
gpu = 0

print("Use GPU: {} for training".format(gpu))

print("=> creating model")

model_finetune = VGG16_HSI()

# Freeze all layers except the last fully connected layer
for param in model_finetune.pre_conv.parameters():
    param.requires_grad = False  # Freeze convolutional layers
for param in model_finetune.fc.parameters():
    param.requires_grad = False  # Freeze convolutional layers
for param in model_finetune.encoder.features.parameters():
    param.requires_grad = False  # Freeze convolutional layers

for param in model_finetune.encoder.classifier[:-1].parameters():
    param.requires_grad = False  # Freeze all but the last FC layer
# Initialize the last FC layer
# Initialize the last FC layer
torch.nn.init.normal_(model_finetune.encoder.classifier[6].weight, mean=0.0, std=0.01)
torch.nn.init.zeros_(model_finetune.encoder.classifier[6].bias)

# Check which layers are trainable
for name, param in model_finetune.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

Use GPU: 0 for training
=> creating model
pre_conv.0.weight: requires_grad=False
pre_conv.0.bias: requires_grad=False
pre_conv.2.weight: requires_grad=False
pre_conv.2.bias: requires_grad=False
pre_conv.3.weight: requires_grad=False
pre_conv.3.bias: requires_grad=False
pre_conv.5.weight: requires_grad=False
pre_conv.5.bias: requires_grad=False
fc.weight: requires_grad=False
fc.bias: requires_grad=False
encoder.features.1.weight: requires_grad=False
encoder.features.1.bias: requires_grad=False
encoder.features.4.weight: requires_grad=False
encoder.features.4.bias: requires_grad=False
encoder.features.6.weight: requires_grad=False
encoder.features.6.bias: requires_grad=False
encoder.features.9.weight: requires_grad=False
encoder.features.9.bias: requires_grad=False
encoder.features.11.weight: requires_grad=False
encoder.features.11.bias: requires_grad=False
encoder.features.13.weight: requires_grad=False
encoder.features.13.bias: requires_grad=False
encoder.features.16.weight: requires_g

In [26]:
print(model_finetune)

VGG16_HSI(
  (pre_conv): Sequential(
    (0): Conv2d(224, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): AdaptiveAvgPool2d(output_size=(1, 1))
  )
  (fc): Linear(in_features=256, out_features=200704, bias=True)
  (encoder): VGG(
    (features): Sequential(
      (0): ReLU(inplace=True)
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): ReLU(inplace=True)
      (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=Tr

In [27]:
test = data_augment[0]
test = torch.tensor(test)
test = test.to(torch.float32)
test = test.unsqueeze(0)

input = test
input = input.permute(0, 3, 1, 2)

test2 = data_augment[1]
test2 = torch.tensor(test2)
test2 = test2.to(torch.float32)
test2 = test2.unsqueeze(0)

input2 = test2
input2 = input2.permute(0, 3, 1, 2)

print(f"input shape: {input.shape}")
print(f"input2 shape: {input2.shape}")

# Pass the input through the model
model_finetune.eval()
output = model_finetune(input)

print(output)

input shape: torch.Size([1, 224, 9, 9])
input2 shape: torch.Size([1, 224, 9, 9])
tensor([[1.8294, 0.6276]], grad_fn=<AddmmBackward0>)


In [28]:
finetune_parameters = sum(p.numel() for p in model_finetune.parameters())
print(f"finetune Parameter {finetune_parameters}")
print(f"Pretrain Parameter {pretrain_parameters}")

finetune Parameter 186401986
Pretrain Parameter 205281472
